In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Read Raw Data


Read the raw data. 

In [2]:
import os 
import pandas as pd

RecName = 'Diag'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)

data/Simulations/Diag.csv


In [3]:

# load all records
df = pd.read_csv(raw_file_path)
df

,PID,ECID,RID,DT,Value
0,P0,EC140,161119,2023/3/19,N91.5
1,P0,EC23,214372,2023/3/19,S53.0
2,P0,EC141,187825,2023/3/20,P24.5
3,P0,EC208,227678,2023/3/20,S21.5
4,P0,EC19,174522,2023/3/20,U59.5
...,...,...,...,...,...
921,P6,EC15,240970,2023/4/7,M43.5
922,P7,EC17,209056,2023/3/18,S37.5
923,P7,EC17,209055,2023/3/18,J24.5
924,P7,EC17,248716,2023/3/24,J74.5


# Preprocess

## Add Missing Records

In [4]:
from recfldgrn.datapoint import load_df_data_from_folder

prefix_cols = ['PID', 'ECID']
focal_cols = ['RID']

Parent_RecName = 'EC'
rec_folder = 'data/ProcData/RecFolder/'
fullrec_folder =  os.path.join(rec_folder, Parent_RecName) 
df_parent = load_df_data_from_folder(fullrec_folder)
df_parent

,PID,ECID,DT_min,DT_max,BasicInfo
0,P0,EC18,2023-03-18,2023-04-16,D&X
1,P0,EC140,2023-03-19,2023-04-05,D&X
2,P0,EC23,2023-03-19,2023-04-16,C&W
3,P0,EC141,2023-03-20,2023-04-16,C&Z
4,P0,EC142,2023-03-20,2023-04-15,B&W
...,...,...,...,...,...
220,P6,EC15,2023-03-21,2023-04-09,B&X
221,P6,EC93,2023-03-23,2023-04-04,B&X
222,P6,EC195,2023-03-25,2023-03-25,C&Z
223,P6,EC173,2023-04-05,2023-04-13,C&W


In [5]:
# this one is try to fill the missing encounter. 
import numpy as np

d = pd.merge(df_parent[prefix_cols], df, on = prefix_cols, how = 'left')
for col in focal_cols: 
    d.loc[d[col].isnull(), col] = np.ones(d[col].isnull().sum()).cumsum() + d[col].max()
    d[col] = d[col].astype(int)
    assert len(d[col]) == d[col].nunique()
    
df = d
df

,PID,ECID,RID,DT,Value
0,P0,EC18,200972,2023/4/10,D14.0
1,P0,EC18,227676,2023/4/13,J19.5
2,P0,EC18,214361,2023/4/16,J74.5
3,P0,EC18,200975,2023/4/8,I94.0
4,P0,EC140,161119,2023/3/19,N91.5
...,...,...,...,...,...
929,P6,EC173,174622,2023/4/5,R80.5
930,P7,EC17,209056,2023/3/18,S37.5
931,P7,EC17,209055,2023/3/18,J24.5
932,P7,EC17,248716,2023/3/24,J74.5


## Update DT Format

In [6]:
df['DT'] = pd.to_datetime(df['DT'])

# Save To Disk by Bucket

In [7]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder

In [8]:
rec_folder = 'data/ProcData/RecFolder'
fullrec_folder = os.path.join(rec_folder, RecName)
print(df.shape, fullrec_folder)

if not os.path.exists(fullrec_folder): os.makedirs(fullrec_folder)
write_df_to_folders(RecName, fullrec_folder, df)


(934, 5) data/ProcData/RecFolder/Diag


In [9]:
from recfldgrn.datapoint import PatientDP
print(rec_folder)
PID = 'P4'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
print(Pat)
dfx = Pat.get_df_rec(RecName)
dfx

data/ProcData/RecFolder


,PID,ECID,RID,DT,Value
0,P4,EC125,238249,2023-03-18,N97.5
1,P4,EC125,158555,2023-03-22,X4.5
2,P4,EC125,158547,2023-04-02,P36.5
3,P4,EC24,238252,2023-03-18,C91.0
4,P4,EC24,145433,2023-03-22,H48.5
...,...,...,...,...,...
79,P4,EC121,158548,2023-04-16,R28.5
80,P4,EC196,248719,NaT,NaN
81,P4,EC203,238255,2023-04-04,P40.5
82,P4,EC204,145439,2023-04-12,U55.0


In [10]:
from recfldgrn.datapoint import load_df_data_from_folder
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder)
df

data/ProcData/RecFolder/Diag


,PID,ECID,RID,DT,Value
0,P0,EC18,200972,2023-04-10,D14.0
1,P0,EC18,227676,2023-04-13,J19.5
2,P0,EC18,214361,2023-04-16,J74.5
3,P0,EC18,200975,2023-04-08,I94.0
4,P0,EC140,161119,2023-03-19,N91.5
...,...,...,...,...,...
929,P6,EC173,174622,2023-04-05,R80.5
930,P7,EC17,209056,2023-03-18,S37.5
931,P7,EC17,209055,2023-03-18,J24.5
932,P7,EC17,248716,2023-03-24,J74.5


# Field & Grain

In [11]:
FieldGrainInfo = []

In [12]:
RecName = 'Diag'
prefix_ids = ['PID', 'ECID']
focal_ids = ['RID']
field_list = ['Value', 'DT']

## Field `Value`

In [13]:
# (a) field and (b) get field-focused df
field = 'Value'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,RID,Value
0,P0,EC18,200972,D14.0
1,P0,EC18,227676,J19.5
2,P0,EC18,214361,J74.5
3,P0,EC18,200975,I94.0
4,P0,EC140,161119,N91.5
...,...,...,...,...
929,P6,EC173,174622,R80.5
930,P7,EC17,209056,S37.5
931,P7,EC17,209055,J24.5
932,P7,EC17,248716,J74.5


### Grain `Diag@Value-DiagDftGrn`

In [14]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info

In [15]:
# (c) grain
grain = 'DiagDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

DiagDftGrn
Value-DiagDftGrn
Diag@Value-DiagDftGrn


In [16]:
# grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
# s = dfx[field].apply(lambda x: grain_str_fn(x))
# s

In [17]:
# Vocab = generate_grain_vocab_info(s) # load from disk
# v2idx = Vocab['v2idx']
# v2idx

In [18]:
# grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
# s = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
# s

In [19]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


{'_padding': 0, '_missing': 1, 'C-5': 2, 'C-0': 3, 'A-R': 4, 'A-U': 5, 'A-Q': 6, 'A-X': 7, 'A-B': 8, 'A-J': 9, 'A-P': 10, 'A-G': 11, 'A-Y': 12, 'A-D': 13, 'A-M': 14, 'A-H': 15, 'A-N': 16, 'A-S': 17, 'A-O': 18, 'A-F': 19, 'A-E': 20, 'A-L': 21, 'A-W': 22, 'A-V': 23, 'A-K': 24, 'A-T': 25, 'A-C': 26, 'A-Z': 27, 'A-A': 28, 'A-I': 29, 'B-M44': 30, 'B-Y41': 31, 'B-G47': 32, 'B-B42': 33, 'B-U55': 34, 'B-H75': 35, 'B-D96': 36, 'B-J74': 37, 'B-O5.': 38, 'B-U84': 39, 'B-Q32': 40, 'B-D33': 41, 'B-G57': 42, 'B-N34': 43, 'B-X4.': 44, 'B-B82': 45, 'B-G91': 46, 'B-J24': 47, 'B-R36': 48, 'B-A94': 49, 'B-G71': 50, 'B-I47': 51, 'B-R37': 52, 'B-N22': 53, 'B-R15': 54, 'B-R12': 55, 'B-X89': 56, 'B-S82': 57, 'B-K25': 58, 'B-W80': 59, 'B-R10': 60, 'B-E71': 61, 'B-J86': 62, 'B-U68': 63, 'B-L91': 64, 'B-P40': 65, 'B-N78': 66, 'B-P48': 67, 'B-K28': 68, 'B-E2.': 69, 'B-B12': 70, 'B-E53': 71, 'B-G2.': 72, 'B-T90': 73, 'B-C30': 74, 'B-P97': 75, 'B-S63': 76, 'B-X86': 77, 'B-U23': 78, 'B-Y38': 79, 'B-P23': 80, 'B-X85

0      [13, 657, 3]
1       [9, 579, 2]
2        [9, 37, 2]
3      [29, 580, 3]
4      [16, 581, 2]
           ...     
929      [4, 85, 2]
930    [17, 800, 2]
931      [9, 47, 2]
932      [9, 37, 2]
933     [14, 30, 2]
Name: Value, Length: 934, dtype: object

In [20]:
series_idx

0      [13, 657, 3]
1       [9, 579, 2]
2        [9, 37, 2]
3      [29, 580, 3]
4      [16, 581, 2]
           ...     
929      [4, 85, 2]
930    [17, 800, 2]
931      [9, 47, 2]
932      [9, 37, 2]
933     [14, 30, 2]
Name: Value, Length: 934, dtype: object

In [21]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list

d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

In [22]:
d

{'RecName': 'Diag',
 'prefix_ids': ['PID', 'ECID'],
 'focal_ids': ['RID'],
 'field_list': ['Value', 'DT'],
 'field': 'Value',
 'grain': 'DiagDftGrn',
 'field2grain': 'Value-DiagDftGrn',
 'recfield2grain': 'Diag@Value-DiagDftGrn',
 'Vocab': {'idx2v': {0: '_padding',
   1: '_missing',
   2: 'C-5',
   3: 'C-0',
   4: 'A-R',
   5: 'A-U',
   6: 'A-Q',
   7: 'A-X',
   8: 'A-B',
   9: 'A-J',
   10: 'A-P',
   11: 'A-G',
   12: 'A-Y',
   13: 'A-D',
   14: 'A-M',
   15: 'A-H',
   16: 'A-N',
   17: 'A-S',
   18: 'A-O',
   19: 'A-F',
   20: 'A-E',
   21: 'A-L',
   22: 'A-W',
   23: 'A-V',
   24: 'A-K',
   25: 'A-T',
   26: 'A-C',
   27: 'A-Z',
   28: 'A-A',
   29: 'A-I',
   30: 'B-M44',
   31: 'B-Y41',
   32: 'B-G47',
   33: 'B-B42',
   34: 'B-U55',
   35: 'B-H75',
   36: 'B-D96',
   37: 'B-J74',
   38: 'B-O5.',
   39: 'B-U84',
   40: 'B-Q32',
   41: 'B-D33',
   42: 'B-G57',
   43: 'B-N34',
   44: 'B-X4.',
   45: 'B-B82',
   46: 'B-G91',
   47: 'B-J24',
   48: 'B-R36',
   49: 'B-A94',
   50: 'B-G7

## Field `DT`

In [23]:
# (a) field and (b) get field-focused df
field = 'DT'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,RID,DT
0,P0,EC18,200972,2023-04-10
1,P0,EC18,227676,2023-04-13
2,P0,EC18,214361,2023-04-16
3,P0,EC18,200975,2023-04-08
4,P0,EC140,161119,2023-03-19
...,...,...,...,...
929,P6,EC173,174622,2023-04-05
930,P7,EC17,209056,2023-03-18
931,P7,EC17,209055,2023-03-18
932,P7,EC17,248716,2023-03-24


### DTDftGrn

In [24]:
# (c) grain
grain = 'DTDftGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

DTDftGrn
DT-DTDftGrn
Diag@DT-DTDftGrn


In [25]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


{'_padding': 0, '_missing': 1, 'Yx': 2, 'Y2010': 3, 'Y2011': 4, 'Y2012': 5, 'Y2013': 6, 'Y2014': 7, 'Y2015': 8, 'Y2016': 9, 'Y2017': 10, 'Y2018': 11, 'Y2019': 12, 'Y2020': 13, 'Y2021': 14, 'Y2022': 15, 'Y2023': 16, 'Y2024': 17, 'Y2025': 18, 'Y2026': 19, 'Y2027': 20, 'Y2028': 21, 'Y2029': 22, 'Mx': 23, 'M1': 24, 'M2': 25, 'M3': 26, 'M4': 27, 'M5': 28, 'M6': 29, 'M7': 30, 'M8': 31, 'M9': 32, 'M10': 33, 'M11': 34, 'M12': 35, 'Dx': 36, 'D1': 37, 'D2': 38, 'D3': 39, 'D4': 40, 'D5': 41, 'D6': 42, 'D7': 43, 'D8': 44, 'D9': 45, 'D10': 46, 'D11': 47, 'D12': 48, 'D13': 49, 'D14': 50, 'D15': 51, 'D16': 52, 'D17': 53, 'D18': 54, 'D19': 55, 'D20': 56, 'D21': 57, 'D22': 58, 'D23': 59, 'D24': 60, 'D25': 61, 'D26': 62, 'D27': 63, 'D28': 64, 'D29': 65, 'D30': 66, 'D31': 67, 'Sx': 68, 'S1': 69, 'S2': 70, 'S3': 71, 'S4': 72, 'Wx': 73, 'W1': 74, 'W2': 75, 'W3': 76, 'W4': 77, 'W5': 78, 'W6': 79, 'W7': 80, 'WYx': 81, 'WY1': 82, 'WY2': 83, 'WY3': 84, 'WY4': 85, 'WY5': 86, 'WY6': 87, 'WY7': 88, 'WY8': 89, 'WY

0      [16, 27, 46, 70, 74, 97, 138]
1      [16, 27, 49, 70, 77, 97, 138]
2      [16, 27, 52, 70, 80, 97, 138]
3      [16, 27, 44, 70, 79, 96, 138]
4      [16, 26, 55, 69, 80, 93, 138]
                   ...              
929    [16, 27, 41, 70, 76, 96, 138]
930    [16, 26, 54, 69, 79, 93, 138]
931    [16, 26, 54, 69, 79, 93, 138]
932    [16, 26, 60, 69, 78, 94, 138]
933    [16, 26, 63, 69, 74, 95, 138]
Name: DT, Length: 934, dtype: object

In [26]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list


d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

# Save Field Grain Info



In [27]:
df_FieldGrainInfo = pd.DataFrame(FieldGrainInfo)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,Diag,"[PID, ECID]",[RID],"[Value, DT]",Value,DiagDftGrn,Value-DiagDftGrn,Diag@Value-DiagDftGrn,"{'idx2v': {0: '_padding', 1: '_missing', 2: 'C..."
1,Diag,"[PID, ECID]",[RID],"[Value, DT]",DT,DTDftGrn,DT-DTDftGrn,Diag@DT-DTDftGrn,"{'idx2v': {0: '_padding', 1: '_missing', 2: 'Y..."


In [28]:
fldgrn_folder = 'data/ProcData/FldGrnInfo'

if not os.path.exists(fldgrn_folder): os.makedirs(fldgrn_folder)

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')
df_FieldGrainInfo.to_pickle(fullfldgrn_file)